In [1]:
!pip install onnx onnxruntime onnxscript
!pip install torch-tensorrt
!pip install --pre -U -f https://mlc.ai/wheels mlc-ai-nightly-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.8/725.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.7 MB/s eta 0:00:00
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.3.0-py2.py3-none-any.whl size=17553 sha256=cd956e7319e27c6a7e89302b84d0282452baf48e5a2e1ba7ccc7798d0d3fee6c
  Stored in directory: /root/.cache/pip/wheels/14/31/c2/a3ecb74def7087b76ef2a701e6823f03e40ab43348c668a571

In [ ]:
if torch.cuda.is_available():
  print("cuda")

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data import *
from imutils import paths
import numpy as np
import random
import cv2
import os
# import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
if torch.cuda.is_available():
    import torch_tensorrt
import time
from PIL import Image, ImageDraw, ImageFont
import torch.nn.utils.prune as prune
import copy
import math
from typing import List
import tvm
from tvm import relax as R
from tvm import te
from tvm import relay

from tvm import autotvm
from tvm.autotvm.tuner import XGBTuner
import tvm.auto_scheduler as auto_scheduler
from tvm.contrib import graph_executor


import time

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
print(tvm.__version__)

0.19.dev0


In [ ]:
class small_basic_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(small_basic_block, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(ch_in, ch_out // 4, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out, kernel_size=1),
        )
        # self.conv1 = nn.Conv2d(ch_in, ch_out // 4, kernel_size=1)
        # self.conv2 = nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(3, 1), padding=(1, 0))
        # self.conv3 = nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(1, 3), padding=(0, 1))
        # self.conv4 = nn.Conv2d(ch_out // 4, ch_out, kernel_size=1)
        # self.relu = nn.ReLU()

    def forward(self, x):
        return self.block(x)
        # x = self.conv1(x)
        # print(x.shape)
        # x = self.relu(x)
        # x = self.conv2(x)
        # print(x.shape)
        # x = self.relu(x)
        # x = self.conv3(x)
        # print(x.shape)
        # x = self.relu(x)
        # x = self.conv4(x)
        return x

class LPRNet(nn.Module):
    def __init__(self, lpr_max_len, phase, class_num, dropout_rate):
        super(LPRNet, self).__init__()
        self.phase = phase
        self.lpr_max_len = lpr_max_len
        self.class_num = class_num
        self.dropout_rate = dropout_rate
        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1), # 0
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),  # 2
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1)),
            small_basic_block(ch_in=64, ch_out=128),    # *** 4 ***
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),  # 6
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2)),
            small_basic_block(ch_in=64, ch_out=256),   # 8
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),  # 10
            small_basic_block(ch_in=256, ch_out=256),   # *** 11 ***
            nn.BatchNorm2d(num_features=256),   # 12
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(4, 1, 2)),  # 14
            nn.Dropout(float(dropout_rate)),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=(1, 4), stride=1),  # 16
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),  # 18
            nn.Dropout(float(dropout_rate)),
            nn.Conv2d(in_channels=256, out_channels=class_num, kernel_size=(13, 1), stride=1), # 20
            nn.BatchNorm2d(num_features=class_num),
            nn.ReLU(),  # *** 22 ***
        )
        self.avg1 = nn.AvgPool2d(kernel_size=5, stride=5)
        self.avg2 = nn.AvgPool2d(kernel_size=(4, 10), stride=(4, 2))
        self.container = nn.Sequential(
            nn.Conv2d(in_channels=448+self.class_num, out_channels=self.class_num, kernel_size=(1, 1), stride=(1, 1)),
            # nn.BatchNorm2d(num_features=self.class_num),
            # nn.ReLU(),
            # nn.Conv2d(in_channels=self.class_num, out_channels=self.lpr_max_len+1, kernel_size=3, stride=2),
            # nn.ReLU(),
        )
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        keep_features = list()
        x = self.quant(x)
        for i, layer in enumerate(self.backbone.children()):
            if i in [3, 7, 14]:
                x = x.unsqueeze(0)
            x = layer(x)
            if i in [3, 7, 14]:
                x = x.squeeze(0)
            if i in [2, 6, 13, 22]: # [2, 4, 8, 11, 22]
                keep_features.append(self.dequant(x))

        global_context = list()
        for i, f in enumerate(keep_features):
            if i in [0, 1]:
                f = self.avg1(f)
            if i in [2]:
                f = self.avg2(f)
            f_pow = torch.pow(f, 2)
            f_mean = torch.mean(f_pow)
            f = torch.div(f, f_mean)
            global_context.append(f)
        x = self.dequant(x)
        x = torch.cat(global_context, 1)
        x = self.container(x)
        logits = torch.mean(x, dim=2)

        return logits

def build_lprnet(lpr_max_len=8, phase=False, class_num=66, dropout_rate=0.5):

    Net = LPRNet(lpr_max_len, phase, class_num, dropout_rate)

    if phase == "train":
        return Net.train()
    else:
        return Net.eval()

In [ ]:
CHARS = ['京', '沪', '津', '渝', '冀', '晋', '蒙', '辽', '吉', '黑',
         '苏', '浙', '皖', '闽', '赣', '鲁', '豫', '鄂', '湘', '粤',
         '桂', '琼', '川', '贵', '云', '藏', '陕', '甘', '青', '宁',
         '新',
         '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
         'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K',
         'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
         'W', 'X', 'Y', 'Z', 'I', 'O', '-'
         ]

CHARS_DICT = {char:i for i, char in enumerate(CHARS)}

class LPRDataLoader(Dataset):
    def __init__(self, img_dir, imgSize, lpr_max_len, PreprocFun=None):
        self.img_dir = img_dir
        self.img_paths = []
        for i in range(len(img_dir)):
            self.img_paths += [el for el in paths.list_images(img_dir[i])]
        random.shuffle(self.img_paths)
        self.img_size = imgSize
        self.lpr_max_len = lpr_max_len
        if PreprocFun is not None:
            self.PreprocFun = PreprocFun
        else:
            self.PreprocFun = self.transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        filename = self.img_paths[index]
        Image = cv2.imread(filename)
        height, width, _ = Image.shape
        if height != self.img_size[1] or width != self.img_size[0]:
            Image = cv2.resize(Image, self.img_size)
        Image = self.PreprocFun(Image)

        basename = os.path.basename(filename)
        imgname, suffix = os.path.splitext(basename)
        imgname = imgname.split("-")[0].split("_")[0]
        label = list()
        for c in imgname:
            # one_hot_base = np.zeros(len(CHARS))
            # one_hot_base[CHARS_DICT[c]] = 1
            label.append(CHARS_DICT[c])

        if len(label) == 8:
            if self.check(label) == False:
                print(imgname)
                assert 0, "Error label ^~^!!!"

        return Image, label, len(label)

    def transform(self, img):
        img = img.astype('float32')
        img -= 127.5
        img *= 0.0078125
        img = np.transpose(img, (2, 0, 1))

        return img

    def check(self, label):
        if label[2] != CHARS_DICT['D'] and label[2] != CHARS_DICT['F'] \
                and label[-1] != CHARS_DICT['D'] and label[-1] != CHARS_DICT['F']:
            print("Error label, Please check!")
            return False
        else:
            return True

In [ ]:
def get_args():
    class Args:
      pass
    args = Args()
    args.cuda = False
    args.num_workers = 8
    args.test_batch_size = 100
    args.show = False
    args.dropout_rate = 0
    args.img_size = [94, 24]
    args.test_img_dirs = "drive/MyDrive/LPRNet_Pytorch/data/test"
    args.lpr_max_len = 8
    args.phase_train = False
    args.pretrained_model = "drive/MyDrive/LPRNet_Pytorch/weights/Final_LPRNet_model.pth"
    args.jit_autotune = False
    return args

In [ ]:
def get_model_size_onnx(model):
    onnx_pgm = torch.onnx.dynamo_export(model, torch.randn(1, 3, 24, 94))
    onnx_pgm.save("model.onnx")
    model_size = os.path.getsize("model.onnx")
    return model_size / (1024 * 1024)

def get_model_size(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width

def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements

def collate_fn(batch):
    imgs = []
    labels = []
    lengths = []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

def test(normal = True, quantize_flag = False, weight_quantize_flag=False, fuse_flag=False,
         sparsity_flag=False, prune_flag=False, jit_compile=False, torch_compile=False,
         tensorrt_compile=False, jit_autotune = False, quant_tensorrt_compile_flag=False):
    args = get_args()
    # device  = torch.device("cpu")
    lprnet = build_lprnet(lpr_max_len=args.lpr_max_len, phase=args.phase_train, class_num=len(CHARS), dropout_rate=args.dropout_rate)
    # args.jit_autotune = jit_autotune

    # if args.jit_autotune:
    #   device = torch.device("cuda:0")
    # lprnet.to(device)
    device = torch.device("cuda:0" if args.cuda else "cpu")
    lprnet.to(device)
    print("Successful to build network!")

    # load pretrained model
    if args.pretrained_model:
        lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))
        print("load pretrained model successful!")
    else:
        print("[Error] Can't found pretrained mode, please check!")
        return False

    test_img_dirs = os.path.expanduser(args.test_img_dirs)
    test_dataset = LPRDataLoader(test_img_dirs.split(','), args.img_size, args.lpr_max_len)
    if fuse_flag:
        lprnet = fuse_lprnet(lprnet)
        print("Fused Model")
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    if quantize_flag:
        qconfig = torch.quantization.get_default_qconfig('x86')
        lprnet.qconfig = qconfig
        lprnet.container.qconfig = None
        lprnet = torch.quantization.prepare(lprnet, inplace=True)
        calibrate_dataset = Subset(test_dataset, range(100))
        print("Running calibration on the lprtnet Model")
        Greedy_Decode_Eval(lprnet, calibrate_dataset, args)
        lprnet = torch.quantization.convert(lprnet, inplace=True)
        print("Quantized Model")
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    if weight_quantize_flag:
        quantizer = WeightQuantizer(bits=8)
        quantized_model = quantizer.quantize_model(lprnet)
        print("Weight Quantized Model")
        Greedy_Decode_Eval(quantized_model, test_dataset, args)

    if sparsity_flag:
        sz = get_model_size(lprnet) / (1024 * 1024 * 8)
        print("Model Size: {} MB".format(sz))
        apply_pruning(lprnet)
        # num_non_pruned_params = get_num_parameters(lprnet, count_nonzero_only=False)
        # # sparsity_1_4(lprnet)
        # sparsity_1_4(lprnet)
        # num_pruned_params = get_num_parameters(lprnet, count_nonzero_only=True)
        # print(f"Total Number of parameters: {num_non_pruned_params}")
        # print(f"Number of pruned parameters: {num_pruned_params}")
        # print(f"Size reduced by {100 * (1 - (num_pruned_params / num_non_pruned_params)):.2f}%")
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    if prune_flag:
        num_non_pruned_params = get_num_parameters(lprnet, count_nonzero_only=False)
        pruner = ChannelPruner(lprnet.backbone, 0.5)
        pruner.apply(lprnet.backbone)
        num_pruned_params = get_num_parameters(lprnet, count_nonzero_only=True)
        print("Pruned Model")
        print(f"Number of non-pruned parameters: {num_non_pruned_params}")
        print(f"Number of pruned parameters: {num_pruned_params}")
        print(f"Size reduced by {100 * (1 - (num_pruned_params / num_non_pruned_params)):.2f}%")
        Greedy_Decode_Eval(lprnet, test_dataset, args)
        # return
    if jit_compile:
        lprnet = torch.jit.optimize_for_inference(torch.jit.script(lprnet.eval()))
        print("JIT Model")
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    if torch_compile:
        lprnet = torch.compile(lprnet)
        print("Torch Model")
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    if tensorrt_compile:
        args.cuda = True
        # inputs = torch_tensorrt.Input(min_shape=[1, 3, *args.img_size[::-1]],
        #                       opt_shape=[args.test_batch_size, 3, *args.img_size[::-1]],
        #                       max_shape=[int(1.5 * args.test_batch_size), 3, *args.img_size[::-1]],
        #                       dtype=torch.float32)
        # lprnet = torch_tensorrt.compile(lprnet.cuda(), ir="dynamo", inputs=inputs, assume_dynamic_shape_support=True)
        torch.compile(lprnet.cuda(), backend="torch_tensorrt", dynamic=False)
        print("TensorRT Model")
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    if quant_tensorrt_compile_flag:
        # args.cuda = True
        quantizer = WeightQuantizer(bits=8)
        quantized_model = quantizer.quantize_model(lprnet)
        print("Weight Quantized Model")
        args.cuda = True
        torch.compile(lprnet.cuda(), backend="torch_tensorrt", dynamic=False)
        print("Quant TensorRT Model")
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    if jit_autotune:
        args.jit_autotune = True
        print("Relay Autotune Model")
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    if normal:
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    # try:
    #     Greedy_Decode_Eval(lprnet, test_dataset, args)
    # finally:
    #     # cv2.destroyAllWindows()
    #     pass

    sz = get_model_size(lprnet, count_nonzero_only=True) / (1024 * 1024 * 8)
    print("Model Size: {} MB".format(sz))

def Greedy_Decode_Eval(Net, datasets, args):
    # TestNet = Net.eval()
    epoch_size = len(datasets) // args.test_batch_size
    batch_iterator = iter(DataLoader(datasets, args.test_batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn))
    if args.jit_autotune:
      module_optim = get_optimized_model(args.test_batch_size, Net)
    Tp = 0
    Tn_1 = 0
    Tn_2 = 0
    t1 = time.time()
    for i in range(epoch_size):
        # load train data
        images, labels, lengths = next(batch_iterator)
        start = 0
        targets = []
        for length in lengths:
            label = labels[start:start+length]
            targets.append(label)
            start += length
        targets = np.array([el.numpy() for el in targets])
        imgs = images.numpy().copy()

        if args.cuda:
            images = Variable(images.cuda())
        else:
            images = Variable(images)

        # forward
        if args.jit_autotune:
          module_optim.set_input("data", images)
          module_optim.run()
          output_shape = (args.test_batch_size, 68, 18)
          prebs = module_optim.get_output(0, tvm.nd.empty(output_shape)).numpy()
        else:
          prebs = Net(images)
          prebs = prebs.cpu().detach().numpy()

        # greedy decode

        preb_labels = list()
        for i in range(prebs.shape[0]):
            preb = prebs[i, :, :]
            preb_label = list()
            for j in range(preb.shape[1]):
                preb_label.append(np.argmax(preb[:, j], axis=0))
            no_repeat_blank_label = list()
            pre_c = preb_label[0]
            if pre_c != len(CHARS) - 1:
                no_repeat_blank_label.append(pre_c)
            for c in preb_label: # dropout repeate label and blank label
                if (pre_c == c) or (c == len(CHARS) - 1):
                    if c == len(CHARS) - 1:
                        pre_c = c
                    continue
                no_repeat_blank_label.append(c)
                pre_c = c
            preb_labels.append(no_repeat_blank_label)
        for i, label in enumerate(preb_labels):
            # show image and its predict label
            if args.show:
                show(imgs[i], label, targets[i])
            if len(label) != len(targets[i]):
                Tn_1 += 1
                continue
            if (np.asarray(targets[i]) == np.asarray(label)).all():
                Tp += 1
            else:
                Tn_2 += 1
        # for i, (pred_label, target) in enumerate(zip(preb_labels, targets)):
        #         print(f"\nSample {i}:")
        #         print(f"Predicted: {pred_label}")
        #         print(f"Target: {target}")

        #         if len(pred_label) != len(target):
        #             print("Length mismatch")
        #             continue
        #         if (np.asarray(target) == np.asarray(pred_label)).all():
        #             print("Correct prediction")
        #         else:
        #             print("Wrong prediction")

    Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
    print("[Info] Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))
    t2 = time.time()
    print("[Info] Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))

def show(img, label, target):
    img = np.transpose(img, (1, 2, 0))
    img *= 128.
    img += 127.5
    img = img.astype(np.uint8)

    lb = ""
    for i in label:
        lb += CHARS[i]
    tg = ""
    for j in target.tolist():
        tg += CHARS[int(j)]

    flag = "F"
    if lb == tg:
        flag = "T"
    # img = cv2.putText(img, lb, (0,16), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.6, (0, 0, 255), 1)
    # img = cv2ImgAddText(img, lb, (0, 0))
    # cv2.imshow("test", img)
    print("target: ", tg, " ### {} ### ".format(flag), "predict: ", lb)
    # cv2.waitKey()
    # cv2.destroyAllWindows()

def cv2ImgAddText(img, text, pos, textColor=(255, 0, 0), textSize=12):
    if (isinstance(img, np.ndarray)):  # detect opencv format or not
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    fontText = ImageFont.truetype("data/NotoSansCJK-Regular.ttc", textSize, encoding="utf-8")
    draw.text(pos, text, textColor, font=fontText)

    return cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR)

Model Test Accuracy and execution without optimizations

In [ ]:
test()

Successful to build network!
load pretrained model successful!


<ipython-input-10-4bbca89c341d>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))
/usr/local/l

[Info] Test Accuracy: 0.899 [899:58:43:1000]
[Info] Test Speed: 0.2635062732696533s 1/1000]
Model Size: 1.7050743103027344 MB


Fusing model

In [ ]:
def fuse_small_basic_block(block):
    # Fuse Conv2d + ReLU layers in the block
    torch.quantization.fuse_modules(block.block, ['0', '1'], inplace=True)
    torch.quantization.fuse_modules(block.block, ['2', '3'], inplace=True)
    torch.quantization.fuse_modules(block.block, ['4', '5'], inplace=True)
    return block

def fuse_lprnet(model):
    # Fuse the layers in the backbone sequentially
    backbone = model.backbone
    torch.quantization.fuse_modules(backbone, ['0', '1', '2'], inplace=True)
    backbone[4] = fuse_small_basic_block(backbone[4])
    backbone[8] = fuse_small_basic_block(backbone[8])
    backbone[11] = fuse_small_basic_block(backbone[11])
    torch.quantization.fuse_modules(backbone, ['16', '17', '18'], inplace=True)
    torch.quantization.fuse_modules(backbone, ['20', '21', '22'], inplace=True)
    return model

Reporting Metrics after Fusing and Post Static Quantization

In [ ]:
test(False,True,True,False,False,False,False,False)

Successful to build network!
load pretrained model successful!
Fused Model


<ipython-input-10-4bbca89c341d>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))
/usr/local/l

[Info] Test Accuracy: 0.896 [896:63:41:1000]
[Info] Test Speed: 0.13891670632362366s 1/1000]
Running calibration on the lprtnet Model


/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


[Info] Test Accuracy: 0.88 [88:6:6:100]
[Info] Test Speed: 0.16175062894821168s 1/100]
Quantized Model
[Info] Test Accuracy: 0.851 [851:99:50:1000]
[Info] Test Speed: 0.02904822301864624s 1/1000]
Model Size: 0.1389923095703125 MB


Performing 1:4 Sparsity based pruning

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune

class OneFourPruning(prune.BasePruningMethod):
    """
    This class implements 1:4 pruning: in each group of 4 weights,
    we keep the largest one and zero out the other three.
    """
    PRUNING_TYPE = "structured_1_4"

    def __init__(self):
        super().__init__()

    def compute_mask(self, tensor, default_mask):
        # First, let's make sure our tensor can be divided into groups of 4
        if tensor.numel() % 4 != 0:
            raise ValueError(f"Tensor has {tensor.numel()} elements which isn't divisible by 4")

        # Reshape the tensor into groups of 4
        original_shape = tensor.shape
        weights_grouped = tensor.view(-1, 4)

        # Find absolute values to compare weight magnitudes
        abs_weights = torch.abs(weights_grouped)

        # For each group of 4, find the indices of the 3 smallest weights
        # We'll keep the largest one and zero out these three
        _, indices = torch.topk(abs_weights, k=1, dim=1, largest=False)

        # Create a mask of ones (keep all weights initially)
        mask = torch.ones_like(weights_grouped)

        # Set the mask to zero for the 3 smallest weights in each group
        mask.scatter_(dim=1, index=indices, value=0)

        # Reshape the mask back to the original tensor shape
        return mask.view_as(tensor) * default_mask

def apply_pruning(model):
    """
    Applies 1:4 pruning to all Conv2d and Linear layers in the model.
    The pruning is made permanent so it persists during training.
    """
    print("Starting pruning process...")

    # Let's keep track of how many parameters we're pruning
    total_params = 0
    pruned_params = 0

    for name, module in model.named_modules():
        # We'll prune both Conv2d and Linear layers
        if isinstance(module, (nn.Conv2d)) and not name.startswith('container'):
            # Count parameters before pruning
            total_params += module.weight.numel()

            # Apply our 1:4 pruning
            prune.custom_from_mask(
                module,
                name='weight',
                mask=OneFourPruning.apply(module, 'weight').compute_mask(
                    module.weight,
                    torch.ones_like(module.weight)
                )
            )
            # OneFourPruning.apply(module, 'weight')

            # Count non-zero parameters after pruning
            nonzero_params = torch.count_nonzero(module.weight).item()
            pruned_params += module.weight.numel() - nonzero_params

            # Make the pruning permanent
            # This ensures the pruning mask is applied to the weights
            # and the pruning persists during training
            prune.remove(module, 'weight')

            print(f"Layer {name}:")
            print(f"  Total parameters: {module.weight.numel()}")
            print(f"  Non-zero parameters: {nonzero_params}")
            print(f"  Sparsity: {100 * (1 - nonzero_params/module.weight.numel()):.2f}%")

    print("\nOverall pruning results:")
    print(f"Total parameters: {total_params}")
    print(f"Pruned parameters: {pruned_params}")
    print(f"Overall sparsity: {100 * pruned_params/total_params:.2f}%")

In [ ]:

test(False,False,False,True,False,False,False,False)

Successful to build network!
load pretrained model successful!
Model Size: 1.705078125 MB
Starting pruning process...
Layer backbone.0:
  Total parameters: 1728
  Non-zero parameters: 1296
  Sparsity: 25.00%
Layer backbone.4.block.0:
  Total parameters: 2048
  Non-zero parameters: 1536
  Sparsity: 25.00%
Layer backbone.4.block.2:
  Total parameters: 3072
  Non-zero parameters: 2304
  Sparsity: 25.00%
Layer backbone.4.block.4:
  Total parameters: 3072
  Non-zero parameters: 2304
  Sparsity: 25.00%
Layer backbone.4.block.6:
  Total parameters: 4096
  Non-zero parameters: 3072
  Sparsity: 25.00%
Layer backbone.8.block.0:
  Total parameters: 4096
  Non-zero parameters: 3072
  Sparsity: 25.00%
Layer backbone.8.block.2:
  Total parameters: 12288
  Non-zero parameters: 9216
  Sparsity: 25.00%
Layer backbone.8.block.4:
  Total parameters: 12288
  Non-zero parameters: 9216
  Sparsity: 25.00%
Layer backbone.8.block.6:
  Total parameters: 16384
  Non-zero parameters: 12288
  Sparsity: 25.00%
Laye

<ipython-input-10-4bbca89c341d>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))
/usr/local/l

[Info] Test Accuracy: 0.686 [686:188:126:1000]
[Info] Test Speed: 0.24449558472633362s 1/1000]
Model Size: 1.31573486328125 MB


Channel Pruning

In [ ]:

def channel_prune(tensor: torch.Tensor, sparsity: float) -> torch.Tensor:

    orig_size = tensor.shape
    tensor = tensor.view(-1)
    num_channels = tensor.shape[0]
    num_pruned_channels = round(num_channels * sparsity)

    channel_norms = tensor.abs()#.sum(dim=(1, 2, 3))  # Calculate L1 norm for each channel
    threshold = channel_norms.kthvalue(num_pruned_channels).values
    mask = torch.gt(channel_norms, threshold).float()
    mask = mask.view(orig_size)
    # mask = mask.unsqueeze(1).unsqueeze(2).unsqueeze(3).expand_as(tensor)
    return mask


class ChannelPruner:
    def __init__(self, model, sparsity_dict):
        self.masks = ChannelPruner.prune(model, sparsity_dict)

    @torch.no_grad()
    def apply(self, model):
        for name, param in model.named_parameters():
            if name in self.masks:
                param *= self.masks[name]

    @staticmethod
    @torch.no_grad()
    def prune(model, sparsity_dict):
        masks = dict()
        for name, param in model.named_parameters():
            if "weight" in name and len(param.shape) == 4:  # Channel pruning for Conv layers
                if isinstance(sparsity_dict, dict):
                    masks[name] = channel_prune(param, sparsity_dict[name])
                else:
                    assert (sparsity_dict < 1 and sparsity_dict >= 0)
                    if sparsity_dict > 0:
                        masks[name] = channel_prune(param, sparsity_dict)
        return masks


In [ ]:
test(normal = False, prune_flag=True)

Successful to build network!
load pretrained model successful!
Pruned Model
Number of non-pruned parameters: 446976
Number of pruned parameters: 242845
Size reduced by 45.67%


<ipython-input-19-4bbca89c341d>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))
/usr/local/l

[Info] Test Accuracy: 0.823 [823:101:76:1000]
[Info] Test Speed: 0.192450138092041s 1/1000]
Model Size: 0.9263801574707031 MB


Weight Quantization

In [ ]:
import torch
import torch.nn as nn
import numpy as np

class WeightQuantizer:
    def __init__(self, bits=8):
        self.bits = bits
        self.qmin = -(2 ** (bits - 1))
        self.qmax = 2 ** (bits - 1) - 1

    def quantize_tensor(self, tensor):
        # Calculate scale factor
        scale = (tensor.max() - tensor.min()) / (self.qmax - self.qmin)

        # Zero point computation
        zero_point = self.qmin - torch.round(tensor.min() / scale)

        # Clip values
        q_tensor = torch.clip(torch.round(tensor / scale + zero_point),
                            self.qmin, self.qmax)

        # Dequantize for forward pass
        return (q_tensor - zero_point) * scale

    def quantize_model(self, model):
        for name, param in model.named_parameters():
            if 'weight' in name:
                param.data = self.quantize_tensor(param.data)
        return model

def test_quantization():
    # Create sample model
    lprnet = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)

    # Initialize quantizer
    quantizer = WeightQuantizer(bits=8)

    # Quantize weights
    quantizer = WeightQuantizer(bits=8)
    quantized_model = quantizer.quantize_model(model)

    return quantized_model

In [ ]:
test(normal=False, weight_quantize_flag=True)

Successful to build network!
load pretrained model successful!
Weight Quantized Model


<ipython-input-49-52679b166827>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))
/usr/local/l

[Info] Test Accuracy: 0.899 [899:65:36:1000]
[Info] Test Speed: 0.04648841619491577s 1/1000]
Model Size: 0.2580413818359375 MB


# MLC Optimization

Autotune using relay and JIT

In [ ]:
target = "llvm"

In [ ]:
def get_optimized_model(test_batch_size, model):
  # model = build_lprnet(lpr_max_len=args.lpr_max_len, phase=args.phase_train, class_num=len(CHARS), dropout_rate=args.dropout_rate)
  input_name = "data"
  input_data = torch.randn(test_batch_size, 3, 24, 94)
  scripted_model = torch.jit.trace(model, input_data).eval()
  shape_list = [(input_name, input_data.shape)]
  mod, params = relay.frontend.from_pytorch(scripted_model, shape_list)
  with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)
  dev = tvm.device(str(target), 0)
  module = graph_executor.GraphModule(lib["default"](dev))

  number = 10
  repeat = 1
  min_repeat_ms = 0  # since we're tuning on a CPU, can be set to 0
  timeout = 10  # in seconds

  # create a TVM runner
  runner = autotvm.LocalRunner(
      number=number,
      repeat=repeat,
      timeout=timeout,
      min_repeat_ms=min_repeat_ms,
      enable_cpu_cache_flush=True,
  )

  tuning_option = {
      "tuner": "xgb",
      "trials": 20,
      "early_stopping": 100,
      "measure_option": autotvm.measure_option(
          builder=autotvm.LocalBuilder(build_func="default"), runner=runner
      ),
      "tuning_records": "autotuning.json",
  }
  tasks = autotvm.task.extract_from_program(mod["main"], target=target, params=params)

  for i, task in enumerate(tasks):
      prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

      # choose tuner
      tuner = "xgb"
      tuner_obj = XGBTuner(task, loss_type="reg")
      tuner_obj.tune(
            n_trial=min(tuning_option["trials"], len(task.config_space)),
            early_stopping=tuning_option["early_stopping"],
            measure_option=tuning_option["measure_option"],
            callbacks=[
                autotvm.callback.progress_bar(tuning_option["trials"],prefix = prefix),
                autotvm.callback.log_to_file(tuning_option["tuning_records"]),
            ],
        )
  with autotvm.apply_history_best(tuning_option["tuning_records"]):
    with tvm.transform.PassContext(opt_level=3, config={}):
        lib = relay.build(mod, target=target, params=params)

  dev = tvm.device(str(target), 0)
  module_optim = graph_executor.GraphModule(lib["default"](dev))
  return module_optim

In [ ]:
#Testing Autotune
test(normal = False,jit_autotune = True)

Successful to build network!
load pretrained model successful!
Relay Autotune Model


<ipython-input-26-255e429946bd>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))
/usr/local/l

[Task  1/13]  Current/Best:   10.09/  15.53 GFLOPS | Progress: (20/20) | 69.75 s Done.
[Task  2/13]  Current/Best:    4.14/  11.94 GFLOPS | Progress: (20/20) | 60.58 s Done.
[Task  3/13]  Current/Best:   12.92/  12.92 GFLOPS | Progress: (20/20) | 80.27 s Done.
[Task  4/13]  Current/Best:    3.43/  16.02 GFLOPS | Progress: (20/20) | 61.47 s Done.
[Task  5/13]  Current/Best:    9.81/  12.75 GFLOPS | Progress: (20/20) | 111.07 s Done.
[Task  6/13]  Current/Best:    9.81/  15.70 GFLOPS | Progress: (20/20) | 51.82 s Done.
[Task  7/13]  Current/Best:   10.76/  14.63 GFLOPS | Progress: (20/20) | 95.38 s Done.
[Task  8/13]  Current/Best:    9.40/  19.32 GFLOPS | Progress: (20/20) | 96.74 s Done.
[Task  9/13]  Current/Best:   13.59/  16.48 GFLOPS | Progress: (20/20) | 131.45 s Done.
[Task 10/13]  Current/Best:    6.38/  18.92 GFLOPS | Progress: (20/20) | 140.17 s Done.
[Task 11/13]  Current/Best:   17.46/  18.20 GFLOPS | Progress: (20/20) | 141.79 s Done.
[Task 12/13]  Current/Best:    0.00/  1

JIT Compilation Optimization

In [ ]:
test(normal = False, jit_compile=True)

Successful to build network!
load pretrained model successful!


<ipython-input-56-c946ab66bb01>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))


JIT Model


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[Info] Test Accuracy: 0.899 [899:60:41:1000]
[Info] Test Speed: 0.24823261880874634s 1/1000]
Model Size: 0.0 MB


Ahead of Time Compilation Optimization

In [ ]:
test(normal = False, torch_compile=True)

Successful to build network!
load pretrained model successful!


<ipython-input-56-c946ab66bb01>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))


Torch Model


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/usr/local/lib/python3.10/dist-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
No CUDA run

[Info] Test Accuracy: 0.898 [898:61:41:1000]
[Info] Test Speed: 0.09941262602806092s 1/1000]
Model Size: 1.1237754821777344 MB


TensorRT Optimization


In [ ]:
test(normal = False, tensorrt_compile=True)

  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))

  warnings.warn(



Successful to build network!
load pretrained model successful!
TensorRT Model
[Info] Test Accuracy: 0.9 [900:58:42:1000]
[Info] Test Speed: 0.025034095525741578s 1/1000]
Model Size: 1.7050743103027344 MB


Combining Weight Quantization and Autotuning

In [ ]:
test(normal=False, quant_tensorrt_compile_flag=True)

  lprnet.load_state_dict(torch.load(args.pretrained_model, map_location=device))

  warnings.warn(



Successful to build network!
load pretrained model successful!
Weight Quantized Model
Quant TensorRT Model
[Info] Test Accuracy: 0.9 [900:62:38:1000]
[Info] Test Speed: 0.012115023374557495s 1/1000]
Model Size: 0.2580413818359375 MB
